In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

C:\Users\karth\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Understanding the data

In [12]:
(x_train,y_train) , (x_test,y_test) = tf.keras.datasets.imdb.load_data()

In [13]:
x_train.shape

(25000,)

In [14]:
len(x_train[0])

218

In [15]:
len(x_train[1])

189

In [48]:
a = [len(x) for x in x_train]
b = [len(y) for y in x_test]
print(max(a),max(b))
c = max(max(a),max(b))

2494 2315


# padding the data to make the data same length

In [37]:
from keras.utils import pad_sequences

In [83]:
x_train_padded = pad_sequences(x_train,padding='post',maxlen=c)
x_test_padded = pad_sequences(x_test,padding='post',maxlen=c)

In [84]:
x_train_padded[0:5]

array([[    1,    14,    22, ...,     0,     0,     0],
       [    1,   194,  1153, ...,     0,     0,     0],
       [    1,    14,    47, ...,     0,     0,     0],
       [    1,     4, 18609, ...,     0,     0,     0],
       [    1,   249,  1323, ...,     0,     0,     0]])

In [51]:
x_train_padded.shape

(25000, 2494)

In [52]:
x_test_padded.shape

(25000, 2494)

In [53]:
y_train.shape

(25000,)

In [44]:
y_train[0:5]

array([1, 0, 0, 1, 0], dtype=int64)

# Defining the model

In [57]:
rnn = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(units=32, input_shape=(c,1),return_sequences=False),
    tf.keras.layers.Dense(units=1,activation='sigmoid'),
])
rnn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 32)                1088      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1121 (4.38 KB)
Trainable params: 1121 (4.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Compiling and fitting the model

In [60]:
rnn.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
rnn.fit(x_train_padded,y_train,epochs=5)

Epoch 1/5
782/782 [==============================] - 496s 632ms/step - loss: 0.6942 - accuracy: 0.4974
Epoch 2/5
782/782 [==============================] - 346s 443ms/step - loss: 0.6941 - accuracy: 0.5000
Epoch 3/5
782/782 [==============================] - 339s 434ms/step - loss: 0.6939 - accuracy: 0.4991
Epoch 4/5
782/782 [==============================] - 360s 461ms/step - loss: 0.6940 - accuracy: 0.5013
Epoch 5/5
782/782 [==============================] - 282s 360ms/step - loss: 0.6938 - accuracy: 0.5007


# Accuracy of 1st model - using word encoding only

In [62]:
rnn.evaluate(x_test_padded,y_test)

782/782 [==============================] - 74s 95ms/step - loss: 0.6932 - accuracy: 0.5000


[0.6931836009025574, 0.5]

# padding for only 100 elements
#### only first 100 elements are taken

In [91]:
x_train_padded1 = pad_sequences(x_train,maxlen=100)
x_test_padded1 = pad_sequences(x_test,maxlen=100)
highest_encoded_word = max(np.max(x_train_padded1),np.max(x_test_padded1))

# Model 2: word embedding 

In [94]:
rnn_word_embed = tf.keras.Sequential([
    tf.keras.layers.Embedding(90000,2,input_length=100),
    tf.keras.layers.SimpleRNN(units=32,return_sequences=False),
    tf.keras.layers.Dense(units=1,activation='sigmoid')
])

rnn_word_embed.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 100, 2)            180000    
                                                                 
 simple_rnn_9 (SimpleRNN)    (None, 32)                1120      
                                                                 
 dense_9 (Dense)             (None, 1)                 33        
                                                                 
Total params: 181153 (707.63 KB)
Trainable params: 181153 (707.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [95]:
rnn_word_embed.compile(
loss='binary_crossentropy',
optimizer='adam',
metrics=['accuracy'])

rnn_word_embed.fit(x_train_padded1,y_train,epochs=5)

Epoch 1/5
782/782 [==============================] - 18s 21ms/step - loss: 0.6764 - accuracy: 0.5576
Epoch 2/5
782/782 [==============================] - 15s 20ms/step - loss: 0.5222 - accuracy: 0.7351
Epoch 3/5
782/782 [==============================] - 15s 19ms/step - loss: 0.3105 - accuracy: 0.8712
Epoch 4/5
782/782 [==============================] - 16s 20ms/step - loss: 0.2143 - accuracy: 0.9185
Epoch 5/5
782/782 [==============================] - 16s 20ms/step - loss: 0.1464 - accuracy: 0.9482


# Evaluating 2nd model 

In [96]:
rnn_word_embed.evaluate(x_test_padded1,y_test)

782/782 [==============================] - 5s 6ms/step - loss: 0.4802 - accuracy: 0.8199


[0.48016291856765747, 0.8198800086975098]